In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date
from datetime import datetime
import re

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# Connect to Google Sheets
scope = ['https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name("/Users/adriansanchezdelasierra/projects/news_parser/new_crawler/newspapers-366612-be359318d4f2.json", scope)
client = gspread.authorize(credentials)

In [18]:
el_diario_sheet = client.open("el_diario").sheet1
el_diario = pd.DataFrame(el_diario_sheet.get_all_records())
#el_diario = pd.read_csv("data/el_diario.csv")
#el_diario.drop("Unnamed: 0",axis=1,inplace=True)

el_diario_bup_sheet = client.open("el_diario_links_bup").sheet1
el_diario_bup_sheet.update([el_diario.columns.values.tolist()] + el_diario.values.tolist())

el_diario.to_csv("data/bups/el_diario_links_bup.csv")
old_links = el_diario.url.to_list()

In [3]:
new_links = pd.DataFrame(columns=["seccion","titulo","url","cmp","idx","dominio","crawl_day"])

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

url = "https://www.eldiario.es/"
driver.get(url)

page_source = driver.page_source

soup = BeautifulSoup(page_source, "html.parser") # or lxml

tags = soup.find_all("h2", {"class": "ni-title"})
errors=[]

for i in tags:
    l=[]
    
    cmp = i.a["cmp-ltrk"]
    idx = i.a["cmp-ltrk-idx"]
    href = i.a["href"]
    if href.startswith("http"):
        link = href
        dominio = href.split("/")[2]
    else:
        link = "https://www.eldiario.es{0}".format(i.a["href"])
        dominio = "www.eldiario.es"
    titulo = i.text
    seccion = i.a["href"].split("/")[1]
    
    l.append([seccion,titulo,link,cmp,idx,dominio,date.today()])
    df_ = pd.DataFrame(l,columns=["seccion","titulo","url","cmp","idx","dominio","crawl_day"])
    new_links = pd.concat([new_links,df_],axis=0)

new_links = new_links[new_links["dominio"] == "www.eldiario.es"]
new_links.reset_index(inplace=True,drop=True)

for col,row in new_links.iterrows():
    if row["url"] in old_links:
        new_links.drop(col,inplace=True)

el_diario = pd.concat([el_diario,new_links],axis=0)
el_diario.reset_index(inplace=True,drop=True)

print("{0} new links will be parsed".format(len(new_links)))

54 new links will be parsed


In [8]:
df_full = pd.DataFrame(columns=["url","seccion","cmp","idx","date","title","sub","sub_2","authors","text","crawl_day"])

for col,row in new_links.iterrows():
    l=[]
    url = row["url"]
    response = requests.get(url)
    if response.status_code != 404:
        soup = BeautifulSoup(response.text, "html.parser")

        if not "ultima-hora" in url.split("/")[4]:
            head = soup.find("head")

            d = soup.find("time",{"class": "date"})
            try:
                date = datetime.strptime(d["datetime"], '%Y-%m-%dT%H:%M:%S+02:00').date()
            except ValueError:
                date = datetime.strptime(d["datetime"], '%Y-%m-%dT%H:%M:%S+01:00').date()
            except TypeError:
                continue

            title = soup.find("h1",{"class": "title"})
            title = title.text.replace("\n"," ")
            title = re.sub(' +', ' ', title)
            
            try:
                ul = soup.find("ul",{"class": "footer"})
                ul = ul.text.replace("\n"," ")
                ul = re.sub(' +', ' ', ul)
                sub = ul.split("—  ")[0]
                try:
                    sub_2 = ul.split("—")[1]
                except IndexError:
                    sub_2 = "no sub_2"
            except AttributeError:
                pass
            
            try:
                author = soup.find("p",{"class": "authors"})
                author = author.text.replace("\n"," ")
            except AttributeError:
                try:
                    author = soup.find("a",{"class": "featured-name"})
                    author = author.text.replace("\n"," ")
                except AttributeError:
                    pass
            t = soup.find_all("p",{"class": "article-text"})
            text = ""

            for i in t:
                text = text + i.text
            text = text.replace("\n"," ")
            text = re.sub(' +', ' ', text)
            
            l.append([row["url"],row["seccion"],row["cmp"],row["idx"], date, title, sub, sub_2, author, text,date.today()])
            df_ = pd.DataFrame(l,columns=["url","seccion","cmp","idx","date","title","sub","sub_2","authors","text","crawl_day"])
            df_full = pd.concat([df_full,df_], axis=0)
    else:
        pass

In [ ]:
df_full["newspaper"] = "el diario"
df_full_ = df_full[["newspaper","title","text"]]
today_news_sheet = client.open("today_news").sheet1
today_news_sheet.update([df_full_.columns.values.tolist()] + df_full_.values.tolist())

In [27]:
df_el_diario_sheet = client.open("el_diario_full").sheet1
df_el_diario = pd.DataFrame(df_el_diario_sheet.get_all_records())
#df_el_diario = pd.read_csv("data/el_diario_full.csv")
#df_el_diario.drop("Unnamed: 0",axis=1,inplace=True)
df_el_diario.reset_index(inplace=True,drop=True)

# ACTUALIZANDO Y EXPORTANDO BACK UP
df_el_diario_bup_sheet = client.open("el_diario_full_bup").sheet1
df_el_diario_bup_sheet.update([df_el_diario.columns.values.tolist()] + df_el_diario.values.tolist())
df_el_diario.to_csv("data/bups/el_diario_full_bup.csv")

# SUMANDO NUEVOS REGISTROS AL TOTAL
df_el_diario = pd.concat([df_el_diario,df_full],axis=0)
df_el_diario.reset_index(inplace=True, drop=True)

# EXPORTANDO A DRIVE Y LOCAL
df_el_diario_sheet.update([df_el_diario.columns.values.tolist()] + df_el_diario.values.tolist())
df_el_diario.to_csv("data/el_diario_full.csv")

# ACTUALIZANDO Y EXPORTANDO OLD LINKS
el_diario.to_csv("data/el_diario.csv")
el_diario_sheet.update([el_diario.columns.values.tolist()] + el_diario.values.tolist())